In [ ]:
import org.apache.spark.streaming.StreamingContext._
import org.apache.spark.streaming.{Seconds, StreamingContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.streaming.Duration
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.{functions => F}
import org.apache.spark.sql.{types => T}
import spark.implicits._

In [ ]:
val df3 = spark.read.format("jdbc").table("houses")

In [ ]:
df3.count

In [ ]:
val df_bronze = spark
    .read
    .format("jdbc")
    .table("houses")

In [ ]:
val df_bronze_1 = df_bronze
    .withColumn("_filename", $"_metadata.file_name")
    .withColumn("_modification_time", $"_metadata.file_modification_time")
    .withColumn("_start", F.lit("").cast(T.TimestampType))
    .withColumn("_end", F.lit("").cast(T.TimestampType))
    .drop("_metadata")
val df_silver_schema = df_bronze_1.schema

In [ ]:
val df_silver_before = spark.catalog.listTables().filter(row => row.name == "silver_house").count match {
    case 0 => spark.createDataFrame(sc.emptyRDD[Row], df_silver_schema)
    case _ => spark.read.schema(df_silver_schema).table("silver.houses")
}

In [ ]:
val new_fields = df_bronze_1.select("data-pk").except(df_silver_before.select("data-pk")).distinct().orderBy($"data-pk")

In [ ]:
val existing_fields = df_silver_before.select("data-pk").except(df_bronze_1.select("data-pk")).distinct().orderBy($"data-pk")